In [407]:
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

from data_loader import get_pdf_filepaths, load_page_and_line_indexes, load_split_data, load_raw_indexes_list
from pdf_reader import parse_pdf
from data_cleaner import clean_initial_indexes, add_split_data
from data_saver import save_page_and_line_indexes, save_split_data, save_raw_indexes_list, save_aggregated_data
from data_transformer import (
  get_candidates_and_frequencies, 
  add_frequencies_column, 
  add_is_in_toc, 
  add_importance,
  add_position_in_context,
  add_is_named_entity,
  add_length_of_word,
  add_is_named_author,
  add_tfidf,
  get_raw_indexes_list,
  add_is_in_index,
  aggregate_by_candidate
)


#InteractiveShell.ast_node_interactivity = "all"

# Configuration variables

In [33]:
DATA_DIR_PATH = "../data/"
PDF_SOURCE_DIR_PATH = DATA_DIR_PATH + "pdf/"
PROCESSED_DATA_DIR_PATH = DATA_DIR_PATH + "processed/"

# Load data

In [34]:
file_paths = get_pdf_filepaths(PDF_SOURCE_DIR_PATH)
file_paths

['/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2017_Book_TheDataScienceDesignManual.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/morris_07.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/lyons_96.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2015_Book_LinearAlgebra.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/christensen_04.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/keefe_00.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/linnebo_18.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/oliver_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/jenkins_08.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/zimmermann_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/shapiro_9

# Raw line and page data


In [474]:

for file_path in file_paths[19:20]:
    raw_line_and_page_indexes = parse_pdf(file_path)

    line_and_page_indexes = clean_initial_indexes(raw_line_and_page_indexes)

  # line_and_page_indexes['file_name']
  # line_and_page_indexes['by_line'].head()
  # line_and_page_indexes['by_page'].head()

    save_page_and_line_indexes(
     processed_data_dir_path=PROCESSED_DATA_DIR_PATH, 
     line_and_page_indexes=line_and_page_indexes
    )


# Data set split


In [475]:
file_path = file_paths[19]

line_and_page_indexes = load_page_and_line_indexes(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)
with_split_data = add_split_data(
  file_path=file_path,
  line_and_page_indexes=line_and_page_indexes
)

# with_split_data['by_page_body'].head()

save_split_data(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  split_data=with_split_data  
)
 


# Prepare input data frames

In [476]:
with_split_data['by_page_toc']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3,clean_content
0,Contents page xi\n\nPreface to the Fifth Edition\nCOMPUTABILITY THEORY\n\n1 Enumerability\n1.1 Enumerability\n1.2 Enumerable Sets\n\n3\n3\n7\n\n2 Diagonalization\n\n16\n\n3 Turing Computability\n\n23\n\n4 Uncomputability\n4.1 The Halting Problem\n4.2 The Productivity Function\n\n35\n35\n40\n\n5 Abacus Computability\n5.1 Abacus Machines\n5.2 Simulating Abacus Machines by Turing Machines\n5.3 The Scope of Abacus Computability\n\n45\n45\n51\n57\n\n6 Recursive Functions\n6.1 Primitive Recursive Functions\n6.2 Minimization\n\n63\n63\n70\n\n7 Recursive Sets and Relations\n7.1 Recursive Relations\n7.2 Semirecursive Relations\n7.3 Further Examples\n\n73\n73\n80\n83\n\n8 Equivalent Definitions of Computability\n8.1 Coding Turing Computations\n8.2 Universal Turing Machines\n8.3 Recursively Enumerable Sets\n\n88\n88\n94\n96 vii,8,8,Contents,,,content page xi preface fifth edition computability theory enumerability enumerability enumerable set diagonalization turing computability uncomputability halting problem productivity function abacus computability abacus machine simulating abacus machine turing machine scope abacus computability recursive function primitive recursive function minimization recursive set relation recursive relation semirecursive relation example equivalent definition computability coding turing computation universal turing machine recursively enumerable set vii
1,"viii\n\nCONTENTS\n\nBASIC METALOGIC\n\n9 A Précis of First-Order Logic: Syntax\n9.1 First-Order Logic\n9.2 Syntax\n\n101\n101\n106\n\n10 A Précis of First-Order Logic: Semantics\n10.1 Semantics\n10.2 Metalogical Notions\n\n114\n114\n119\n\n11 The Undecidability of First-Order Logic\n11.1 Logic and Turing Machines\n11.2 Logic and Primitive Recursive Functions\n\n126\n126\n132\n\n12 Models\n12.1 The Size and Number of Models\n12.2 Equivalence Relations\n12.3 The Löwenheim–Skolem and Compactness Theorems\n\n137\n137\n142\n146\n\n13 The Existence of Models\n13.1 Outline of the Proof\n13.2 The First Stage of the Proof\n13.3 The Second Stage of the Proof\n13.4 The Third Stage of the Proof\n13.5 Nonenumerable Languages\n\n153\n153\n156\n157\n160\n162\n\n14 Proofs and Completeness\n14.1 Sequent Calculus\n14.2 Soundness and Completeness\n14.3 Other Proof Procedures and Hilbert’s Thesis\n\n166\n166\n174\n179\n\n15 Arithmetization\n15.1 Arithmetization of Syntax\n15.2 Gödel Numbers\n15.3 More Gödel Numbers\n\n187\n187\n192\n196\n\n16 Representability of Recursive Functions\n16.1 Arithmetical Definability\n16.2 Minimal Arithmetic and Representability\n16.3 Mathematical Induction\n16.4 Robinson Arithmetic\n\n199\n199\n207\n212\n216\n\n17 Indefinability, Undecidability, Incompleteness\n17.1 The Diagonal Lemma and the Limitative Theorems\n17.2 Undecidable Sentences\n17.3 Undecidable Sentences without the Diagonal Lemma\n\n220\n220\n224\n226\n\n18 The Unprovability of Consistency\n\n232",9,9,Contents,,,viii content basic metalogic first-order logic syntax first-order logic syntax first-order logic semantics semantics metalogical notion undecidability first-order logic logic turing machine logic primitive recursive function model size number model equivalence relation compactness theorem existence model outline proof first stage proof second stage proof third stage proof nonenumerable language proof completeness sequent calculus soundness completeness proof procedure hilbert thesis arithmetization arithmetization syntax number number representability recursive function arithmetical definability minimal arithmetic representability mathematical induction robinson arithmetic indefinability undecidability incompleteness diagonal lemma limitative theorem undecidable sentence undecidable sentence without diagonal lemma unprovability consistency
2,CONTENTS ix\n\nFURTHER TOPICS\n\n19 Normal Forms\n19.1 Disjunctive and Prenex Normal Forms\n19.2 Skolem Normal Form\n19.3 Herbrand’s T

In [477]:
with_split_data['by_page_biblio']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3


In [478]:
with_split_data['by_page_index']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3
0,"Index abacus (machine), 45ff, simulation of Turing machine by, 51ff abacus-computable function, 46ff abbreviation, 108ff accent (! ), 64\nAckermann, Wilhelm, see Ackermann function\nAckermann function, 84f\nA-correct, 290\nAddison, John, see Addison’s theorem\nAddison’s theorem, 286, 289ff addition, abacus computability of, 48f, laws of, 218,\nTuring computability of, 29f address of a register, 46 ampersand (&), 102 analysis, 312ff, axiomatic, 314, predicative, 315 analytical sets, relations, functions, 284 antidiagonal sequence, 18, set, 18\nArabic numerals, see decimal representation argument(s) of a function, 4\nAristotle, 272 arithmetic (true arithmetic), 150, 207, non-standard models of, 150f, 302ff, undecidability of, 222, without addition, 295, without multiplication, 295; see also P, Q, R arithmetical classes, 287 arithmetical completeness theorem, see Solovay completeness theorem arithmetical definability, see arithmetical sets, arithmetical classes arithmetical equivalence of formulas, 205 arithmetical Löwenheim–Skolem theorem, 305, 317f arithmetical sets and relations, 199, 286ff arithmetical soundness theorem, 335 arithmetization of syntax, 187ff arrow (→), 102, 107f, 327 associative laws of addition and multiplication, 218, of conjunction and disjunction, 245 atomic formula, 107 atomic term, 107 avoidable appeal to Church’s thesis, 83 axiom of choice (AC), 163, 248, 341 axiom of enumerability, 281, of induction, 214, 283,\n314, of infinity, 282 axioms of GL, 333, of K, 328, of P, 214–215, of Q,\n207f, of R, 216, of S5, 340 axiom scheme, 214 axiomatizable theory, 191, finitely, 191 back-and-forth argument, 345 bars, 262\nBarwise, Jon, 341 base-b representation of numbers, 11; see also binary, decimal, duodecimal base step in proof by induction, 109, 212–213 basic functions, 64\nBehmann, Heinrich, see Löwenheim–Behmann theorem\nBenacerraf, Paul, xii\nBernays, Paul, 233\nBerry’s paradox, 227\nBertrand’s postulate, see Chebyshev’s theorem beta function (β-function) lemma, 203\nBeth, Ewart W., see Beth definability theorem\nBeth definability theorem, 265ff biconditional (↔), 102, 108 binary representation of numbers, 11, 21, 89\nBoole, George, 272 box (!, !), 328, 332 box of an abacus, 46 bound variables, 111, relettering of, 124 bounded minimization and maximization, 77 bounded quantification, 76 branch, 323\nBüchi, J. R., see Turing–Büchi proof busy beaver problem, see productivity\n"" canonical domains, 142, 147\nCantor, Georg, 239; see also back-and-forth argument,\nCantor’s theorem, diagonalization method, zig-zag enumeration\nCantor’s theorem, 16ff caret (∧), 328\n\n343",358,358,Index,,
1,"344\n\nINDEX categorical theory, see denumerably categorical\nChaitin, Gregory, see Chaitin’s theorem\nChaitin’s theorem, 228f characteristic function, 73\nChebyshev’s theorem, 204, 238\nCh’in Chiu-shiao (Qin Jiushao), see Chinese remainder theorem\nChinese remainder theorem, 203 choice, axiom of, see axiom of choice\nChurch, Alonzo, 239; see also Church–Herbrand theorem, Church’s theorem, Church’s thesis\nChurch–Herbrand theorem, 270f; see also dyadic logic\nChurch’s theorem, 120, 132, 134, Gödel-style proof of, 132ff, Turing–Büchi proof of, 126ff\nChurch’s thesis, 71, 134, 189, 192, avoidable and unavoidable appeals to, 83, extended, 71 class, 286 clique, 319 closed formula or sentence, 103, 112 closed term, 103 closure properties of recursive relations, 76, of semi-recursive relations, 81f closure properties of a set of sentences, 155 code number, 8ff, of an expression, 188, 193, of a sequence, 12f, of a Turing machine, 36ff coding operations of a Turing machine, 88ff; see also code number coextensive, 296 cofinite, 15\nCohen, Paul, 239 coherence, 301 combining Turing machines, 39 commutative laws of addition and multiplication, 218, of conjunction and disjunction, 245 compactness theorem, 137, 147ff, and second-order logic, 279, 283,

In [479]:


split_data = load_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
# split_data['by_line_toc'].head()

(candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

# for file_path in file_paths[3:4]:
#   split_data = load_split_data(
#     processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
#     pdf_filepath=file_path
#   )
#   split_data['by_line_toc'].head()

#   (candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

#   candidates_df.head()

#   with_frequencies = add_frequencies_column(
#     by_pages_body_df=split_data['by_page_body'],
#     candidates_df=candidates_df,
#     freq_ngrams=freq_ngrams
#   )

#   with_frequencies.head()

#candidates_df.head(30)

In [480]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [481]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

#with_is_in_toc.head(30)


In [482]:
with_position_in_context = add_position_in_context(with_is_in_toc)



In [483]:
with_importance = add_importance(with_position_in_context)

In [484]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [485]:
with_length_of_word = add_length_of_word(candidates_df)

In [486]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_of_word,
  df_pages_biblio=split_data['by_page_biblio']
)

In [487]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_pages_body=split_data['by_page_body']
)

In [488]:
with_tfidf.head()

,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc,importance,position_in_context,is_named_entity,length,is_named_author,tfidf
0,enumerability,enumerability,enumerability,NOUN,0.000110,1,0.336121,0.000000,0,13,0,0.095567
1,ultimate,ultimate goal present celebrated theorem inherent limit computed proved,our ultimate goal will be to present some celebrated theorem about inherent limit on what can be computed and on what can be proved,ADJ,0.000006,0,0.794533,0.043478,0,8,0,0.022283
2,goal,ultimate goal present celebrated theorem inherent limit computed proved,our ultimate goal will be to present some celebrated theorem about inherent limit on what can be computed and on what can be proved,NOUN,0.000061,0,0.794533,0.086957,0,4,0,0.035731
3,present,ultimate goal present celebrated theorem inherent limit computed proved,our ultimate goal will be to present some celebrated theorem about inherent limit on what can be computed and on what can be proved,NOUN,0.000579,0,0.794533,0.260870,0,7,0,0.091118
4,celebrated,ultimate goal present celebrated theorem inherent limit computed proved,our ultimate goal will be to present some celebrated theorem about inherent limit on what can be computed and on what can be proved,VERB,0.000024,0,0.794533,0.347826,0,10,0,0.049987


In [489]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [490]:
raw_indexes_list

[['abacus',
  'machine',
  'ff',
  'simulation turing',
  'machine',
  'ff abacus',
  'computable function',
  'ff abbreviation',
  'ff accent',
  'abacus-computable'],
 ['ackermann', 'wilhelm', 'ackermann function'],
 ['ackermann function', 'f'],
 ['', 'correct'],
 ['addison', 'john', 'addison', 'theorem'],
 ['addison', 'theorem', 'ff addition', 'abacus computability', 'f', 'law'],
 ['turing computability',
  'f address',
  'register',
  'ampersand',
  'analysis',
  'ff',
  'axiomatic',
  'predicative',
  'analytical set',
  'relation',
  'function',
  'antidiagonal sequence',
  'set'],
 ['arabic numeral', 'decimal representation', 'argument', '', 'function'],
 ['aristotle',
  'arithmetic',
  'true arithmetic',
  'non',
  'standard model',
  'f',
  'ff',
  'undecidability',
  'without addition',
  'without multiplication',
  'p',
  'q',
  'r arithmetical class',
  'arithmetical completeness theorem',
  'solovay completeness',
  'theorem arithmetical definability',
  'arithmetical set'

In [491]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [492]:
# clean_indexes


In [493]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [494]:
with_is_in_index = add_is_in_index(
  candidates_df=with_tfidf,
  indexes_list=clean_indexes
)

In [495]:
aggregated_df=aggregate_by_candidate(
  candidates_df=with_is_in_index  
)

In [496]:
print(len(aggregated_df.candidate_keyword.unique()))
aggregated_df.shape[0]

36550


36550

In [497]:
#CHANGEEEEE
save_aggregated_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    agg_df=aggregated_df,
    file_name="2007BoolosComputability and Logic Fifth Edition.pdf"
)

In [498]:
aggregated_df[aggregated_df.is_in_index==1].head(100)

,candidate_keyword,length,is_named_entity,is_named_author,is_in_toc,freq,is_in_index,tfidf,importance,position_in_context,POS
13,abacus,6,1,0,1,0.000481,1,0.625571,0.468526,0.305833,NOUN
14,abacus computability,20,0,0,0,0.000091,1,0.000000,0.474793,0.159890,CHUNK
30,abacus-computable,17,0,0,0,0.000049,1,0.000000,0.650504,0.491895,NOUN
50,abbreviation,12,0,0,0,0.000146,1,0.052734,0.521706,0.620323,NOUN
110,abstract,8,0,0,0,0.000043,1,0.031532,0.588718,0.443854,ADJ
126,ac,2,0,0,0,0.000006,1,0.098721,0.387105,1.000000,NOUN
234,ackermann,9,1,0,0,0.000006,1,0.042730,0.400830,0.666667,PROPN
235,ackermann function,18,0,0,0,0.000006,1,0.000000,0.400830,0.666667,CHUNK
381,addison,7,1,0,0,0.000024,1,0.079109,0.611739,0.520022,PROPN
383,addition,8,0,0,0,0.000512,1,0.125900,0.630070,0.492928,NOUN
